Importing the necessary libraries for the analysis:

In [59]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series,DataFrame

Pointing the requests library to the content of the Wikipedia page and viewing the pages (commented out)

In [60]:
r=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
c=r.content
#c

In [61]:
soup=BeautifulSoup(c,"html.parser")
#print(soup.prettify())

Finding the table on the Wikipedia page, extracting information from the rows and excluding the header

In [62]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [63]:
#replacing the \n seen from viewing the table information in some cells

rows = table.find_all('tr')[1:]
rows[0].find_all('td')[2].text.replace("\n","")

'Not assigned'

In [64]:
#Viewing what the first cell looks like

rows[0]

<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>

In [65]:
#Creating 3 series for the 3 columns and appending extracted data to them

PostalCode = []
Borough = []
Neighborhood = []


for item in rows:
    PostalCode.append(item.find_all('td')[0].text.replace("\n",""))
    Borough.append(item.find_all('td')[1].text.replace("\n",""))
    Neighborhood.append(item.find_all('td')[2].text.replace("\n",""))
    

In [66]:
#Creating pandas series from the lists

PostalCode = Series(PostalCode)
Borough = Series(Borough)
Neighborhood = Series(Neighborhood)

In [67]:
#Concatenating all three series to a pandas DataFrame

df = pd.concat([PostalCode,Borough,Neighborhood],axis=1)

In [68]:
#adding columns to DataFrame

df.columns = ['PostalCode','Borough','Neighborhood']
#df

In [69]:
df.shape

(288, 3)

In [70]:
#Creating new DataFrame ignoring boroughs that are not assigned

df_new = df[df['Borough']!='Not assigned'].reset_index(drop=True)

In [71]:
df_new.shape

(211, 3)

In [72]:
#Viewing the df_new DataFrame (commented out)
#df_new

In [73]:
#Using groupby Postcode as well as applying lambda function to create new DataFrame

df_newer = df_new.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()

In [74]:
#Viewing all neighborhoods designated as 'Not assigned'

df_newer[df_newer['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [75]:
#Setting neighborhood that are 'Not assigned' to same value as Borough

df_newer.loc[df_newer['Neighborhood']=='Not assigned', ['Neighborhood']] = df_newer['Borough']

In [76]:
#Checking that it worked

df_newer[df_newer['PostalCode']=='M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [19]:
#Viewing the latest version of DataFrame

#df_newer

In [20]:
df_newer.shape

(103, 3)

In [97]:
df_coor = pd.read_csv('Geospatial_Coordinates.csv')

In [104]:
df_coor.rename(index=str, columns={"Postal Code": "PostalCode"},inplace=True)


In [105]:
df_coor.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [112]:

df_full = pd.merge(df_newer,df_coor,on='PostalCode')

In [114]:
df_full.shape

(103, 5)